In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import time

In [3]:
def conv_layer(input, size_in, size_out, name="Conv_Scope"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W_Conv_Scope")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B_Conv_Scope")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("Weights", w)
        tf.summary.histogram("Biases", b)
        tf.summary.histogram("Activations", act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [4]:
def fc_layer(input, size_in, size_out, name="FC_Scope"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W_FC_Scope")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B_FC_Scope")
        act = tf.matmul(input, w) + b
        tf.summary.histogram("Weights", w)
        tf.summary.histogram("Biases", b)
        tf.summary.histogram("Activations", act)
        return act

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')
x_train /= 255
x_test /= 255

In [7]:
x = tf.placeholder(tf.float32, shape=[None, 784], name="Input_X")
x_image = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image('Sample_Image', x_image, 5)

y = tf.placeholder(tf.int64, shape=[None,], name="Output_Y")

In [8]:
conv1 = conv_layer(x_image, 1, 32, name="Conv_Layer_1")
conv_out = conv_layer(conv1, 32, 64, name="Conv_Layer_2")

In [9]:
flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64], name="Flattened_Layer")

In [10]:
fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, name="FC_Layer")
relu = tf.nn.relu(fc1, name="Relu_Final")
tf.summary.histogram("Fc1/Relu", relu)

<tf.Tensor 'Fc1/Relu:0' shape=() dtype=string>

In [11]:
logits = fc_layer(relu, 1024, 10, name="Logits_Results")
softmax = tf.nn.softmax(logits, name="Logits_Softmax")

In [12]:
with tf.name_scope("Accuracy_Scope"):
    matches = tf.equal(tf.argmax(softmax, 1), y, name="Matches_")
    acc = tf.reduce_mean(tf.cast(matches, tf.float32), name="Accuracy_Train")
    tf.summary.scalar("Accuracy_Value", acc)

In [13]:
with tf.name_scope("Loss_Scope"):
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y), name="Loss_Function")
    tf.summary.scalar("Loss_Value", loss)

In [14]:
with tf.name_scope("Train_OP_Scope"):
    train_op = tf.train.AdamOptimizer(0.001).minimize(loss)

In [15]:
mergerd_summary = tf.summary.merge_all()

In [16]:
init = tf.global_variables_initializer()

In [17]:
saver = tf.train.Saver()

In [18]:
with tf.Session() as sess:
    sess.run(init)
    
    writer = tf.summary.FileWriter("./log/CNN_TB/Round1")
    writer.add_graph(sess.graph)

    losses = []
    accuracies = []
    epochs = 5
    batch_size = 1000
    
    for i in range(epochs):
        start_time = time.time()
        
        for curr_batch in range(len(x_train) // batch_size):
            from_i = curr_batch * batch_size
            to_i = (curr_batch * batch_size) + batch_size
            
            loss_, _, acc_, summ = sess.run([loss, train_op, acc, mergerd_summary],
                                      feed_dict={x: x_train[from_i : to_i].reshape(batch_size, 784), 
                                                 y: y_train[from_i : to_i]})
            
        writer.add_summary(summ, i)  
        losses.append(loss_)
        accuracies.append(acc_)
        
        print("Epoch No: " + str(i + 1) + " of " + str(epochs))
        print("Loss: " + str(loss_) + " Accuracy: " + str(acc_))
        
        elapsed_time = time.time() - start_time
        print(time.strftime("%S", time.gmtime(elapsed_time)) + " Seconds")
        
        print("\n")
        
    saver.save(sess, "./log/CNN_TF_Tensorboard.ckpt")
    print("Finish Training :-)")

Epoch No: 1 of 5
Loss: 0.122876726 Accuracy: 0.972
00 Seconds


Epoch No: 2 of 5
Loss: 0.06737193 Accuracy: 0.989
59 Seconds


Epoch No: 3 of 5
Loss: 0.05339623 Accuracy: 0.992
59 Seconds


Epoch No: 4 of 5
Loss: 0.048450682 Accuracy: 0.992
59 Seconds


Epoch No: 5 of 5
Loss: 0.04075613 Accuracy: 0.993
59 Seconds


Finish Training :-)


# tensorboard --logdir log/CNN_TB --port=6007